In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

In [2]:
def get_filepath_wtout_ext(file_wt_full_path):
    #/a/b/c.csv => /a/b/c
    return os.path.splitext(file_wt_full_path)[0]

In [3]:
def get_filename(file_wt_full_path):
    filepath_wout_ext = get_filepath_wtout_ext(file_wt_full_path)
    #/a/b/c => c
    return ntpath.basename(filepath_wout_ext)


In [4]:
def get_filepath(file_wt_full_path):
    #/a/b/c.csv => /a/b
    return os.path.dirname(file_wt_full_path)

In [5]:
def get_file(file_wt_full_path):
    #/a/b/c.csv => c.csv
    return os.path.basename(file_wt_full_path)

## Enter full path to xml file

In [6]:
## Enter full path to xml file
#xml_file = 'Posts*.xml'

#common_path = '../../../../stackexchange_v2/workspace/input'
common_path = '../SO_data_dump'
relative_path = 'posts/Posts*.csv'

filepath = '{}/{}'.format(common_path, relative_path)

In [7]:
#filepath=input()

## Read csv file

In [8]:
ddf = dd.read_csv(filepath, engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)

In [9]:
ddf

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,CommunityOwnedDate,ContentLicense,ParentId,DeletionDate,OwnerDisplayName,ClosedDate
npartitions=9354,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
ddf.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId',
       'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title',
       'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
       'CommunityOwnedDate', 'ContentLicense', 'ParentId', 'DeletionDate',
       'OwnerDisplayName', 'ClosedDate'],
      dtype='object')

In [11]:
ddf.npartitions

9354

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [12]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

In [13]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.166:37768 Dashboard: http://192.168.94.166:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [14]:
client.get_versions(check=True)

{'scheduler': {'host': {'python': '3.9.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '3.10.0-693.2.2.el7.x86_64',
   'machine': 'x86_64',
   'processor': 'x86_64',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'en_US.UTF-8'},
  'packages': {'python': '3.9.5.final.0',
   'dask': '2021.04.1',
   'distributed': '2021.04.1',
   'msgpack': '1.0.2',
   'cloudpickle': '1.6.0',
   'tornado': '6.1',
   'toolz': '0.11.1',
   'numpy': '1.20.2',
   'lz4': None,
   'blosc': None}},
 'workers': {},
 'client': {'host': {'python': '3.9.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '3.10.0-693.2.2.el7.x86_64',
   'machine': 'x86_64',
   'processor': 'x86_64',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'en_US.UTF-8'},
  'packages': {'python': '3.9.5.final.0',
   'dask': '2021.04.1',
   'distributed': '2021.04.1',
   'msgpack': '1.0.2',
   'cloudpickle': '1.6.0',
   'tornado': '6.1',
   'toolz': '0.11.1',
   'numpy': '1.20.2',
   'l

### Get the total number of posts

In [15]:
init_posts_len = ddf.index.shape[0].compute() # compute length of posts_df
print('Total number of posts {}'.format(init_posts_len))

54741615


### Get length of each partition and sum them up
- faster way to compute length

In [16]:
#returns a Series of the length of each partition
#len_of_ppt_ser=ddf.map_partitions(len).compute()

### Sum up each partition

In [17]:
#sum up the Series of length
#init_posts_len = pd.Series(len_of_ppt_ser, dtype="int64").sum()

## Drop the rows with NaN in the Tag column 

In [18]:
ddf = ddf.dropna(subset=['Tags']) # drop the nan in 'Tags' column

## Get the total number of posts with Tags

In [19]:
posts_drp_nan_len = ddf.index.shape[0].compute() # compute length of posts_df
print('Total number of posts with Tags {}'.format(posts_drp_nan_len))

21978328


## Get the total number of posts droped

In [20]:
print(init_posts_len - posts_drp_nan_len)

32763287


## Get the posts with java tags

In [21]:
ddf = ddf[ddf['Tags'].str.contains('<java>', case=False, regex=True)]

### Get the total number of the posts with the java tags

In [22]:
posts_len_java_tags = ddf.index.shape[0].compute()
print('Total number of java posts {}'.format(posts_len_java_tags))

1813938


## Make a folder in that directory

In [23]:
## Make a folder in that directory
folder = '{}/javaposts_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [24]:
print(folder)

../SO_data_dump/javaposts_csv


## Save files in that directory

In [25]:
## Save files in that directory
file = 'JavaPosts'
_ = ddf.to_csv('{}/{}*.csv'.format(folder, file), sep=',', index=False)

distributed.scheduler - ERROR - Couldn't gather keys {'list-6b65ee73-2ea0-44d3-acb7-393ae330de21': ['tcp://10.65.4.92:39166']} state: ['waiting'] workers: ['tcp://10.65.4.92:39166']
NoneType: None
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://10.65.4.92:39166'], list-6b65ee73-2ea0-44d3-acb7-393ae330de21
NoneType: None
